#### Comparative Similarity Estimation
What are we Doing? This is essentially a fact finding experiment.

We want to estimate the possibility of using the current corpus as part of a model to evaluate potential future articles
<div>
   <img src="../../../images/BERTScoring3.png" height="374"; width="374">
</div>

In [1]:
# LIBRARY_PATH = '/home/deleidos/Notebooks/Newspaper/SemanticEstimation'
LIBRARY_PATH = "/home/deleidos/Notebooks/lib"

import sys
import os

module_path = os.path.abspath(os.path.join(LIBRARY_PATH))
if module_path not in sys.path:
    sys.path.append(module_path)
print (f"sys.path : {sys.path}") 


sys.path : ['/home/deleidos/Notebooks/Newspaper/SemanticEstimation/BERTScore', '/opt/conda/lib/python38.zip', '/opt/conda/lib/python3.8', '/opt/conda/lib/python3.8/lib-dynload', '', '/opt/conda/lib/python3.8/site-packages', '/opt/conda/lib/python3.8/site-packages/IPython/extensions', '/home/deleidos/.ipython', '/home/deleidos/Notebooks/lib']


### Preparation

Note, we are now using the practice of centralising python libraries specific to this set of Jupyter Notebooks in one location. This is possible because Python uses the concept of module naming and location as shown in the graphic below
<div>
   <img src="../../../images/SysPath.jpg" height="374"; width="374">
</div>

In [2]:
#!pip install sentence_transformers
import torch

import os, csv
from csv import writer, DictWriter
from os import listdir
import os.path as osp
from os.path import isfile, isdir, join

import time
import datetime
from datetime import datetime as DT
import json
import random
# setup widget import
from ipywidgets import widgets,Layout
from IPython.display import display

import nltk
# nltk.download('punkt')
# nltk.download('brown')
# Locals 
from pathlib import Path 
import os, sys 
import glob

from IPython.display import Image
from IPython.core.display import HTML 
from sentence_transformers import SentenceTransformer, util

# nltk.download('punkt')
###
module_path = str(Path.cwd().parents[0] / "")
if module_path not in sys.path:
    sys.path.append(module_path)  
print (f"sys.path : {sys.path}") 

import random
from timeit import default_timer as timer
import time
import datetime
from datetime import datetime as DT
from os import listdir
import os.path as osp
from os.path import isfile, join

# setup widget import
from ipywidgets import widgets,Layout
from IPython.display import display

import pandas as pd
import numpy as np
# regex
import re
p = re.compile('[a-z]+')
p
re.compile('[a-z]+')
## LOCAL 
CSVPath = "/home/deleidos/Notebooks/CSVData"
JSONPath = "/home/deleidos/Notebooks/JSONData"

from APIutils import *
from dynamoDButils import *
# from utils import *

###
def complete(state="complete"):
    print(f"\nCell {state} @ {(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))}")

complete(state='imports done')


sys.path : ['/home/deleidos/Notebooks/Newspaper/SemanticEstimation/BERTScore', '/opt/conda/lib/python38.zip', '/opt/conda/lib/python3.8', '/opt/conda/lib/python3.8/lib-dynload', '', '/opt/conda/lib/python3.8/site-packages', '/opt/conda/lib/python3.8/site-packages/IPython/extensions', '/home/deleidos/.ipython', '/home/deleidos/Notebooks/lib', '/home/deleidos/Notebooks/Newspaper/SemanticEstimation']

Cell imports done @ 2023-02-08 19:35:09


[nltk_data] Downloading package punkt to /home/deleidos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
jsonfiles = [f for f in listdir(JSONPath) if isfile(join(JSONPath, f))]
jsonfiles.sort()
jsonfiles = [x for x in jsonfiles if (os.path.splitext(x)[1][1:]).lower() == "json"]
jsnWD = widgets.Select(
    options=jsonfiles,
    value=jsonfiles[0],
    description='Choose ID:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        layout=Layout(width='50%', height='256px')
)
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
#         print ("changed to %s" % change['new'])
        selection = change['new']

jsnWD.observe(on_change)
display(jsnWD)

Select(description='Choose ID:', layout=Layout(height='256px', width='50%'), options=('2000CWE.json', 'NOTsema…

In [16]:
jsnf = jsnWD.value
# probable source jsnf='semanticSimArticlesTestingDF.json' 
print (f"{jsnf=} ")

jsnf='semanticSimArticlesTestingDF.json' 


In [17]:
jsnfile = osp.join (JSONPath,jsnf)
df = pd.read_json(jsnfile)
# df = df.transpose()

In [19]:
df

,article,summary,hf1,hf2,hf3,hf4,hf5,hfAvg,kb1,kb2,kb3,kb4,kb5,kbAvg,topic1,score1,topic2,score2
0,https://datainnovation.org/2022/11/visualizing...,economist created visualization showing politi...,0.5627,0.5556,0.5153,0.5132,0.4939,0.528140,0.408926,0.408615,0.395756,0.385963,0.365358,0.392923,economics,0.629220,politics,0.167246
1,https://venturebeat.com/ai/stability-ai-looks-...,among organization building generative technol...,0.9986,0.6797,0.6685,0.6572,0.6319,0.659325,0.515490,0.456996,0.455971,0.445295,0.442166,0.450107,technology,0.585379,innovation,0.180912
2,https://venturebeat.com/ai/openai-ceo-admits-c...,watch demand session today watch herealtman la...,0.5767,0.5576,0.5529,0.5504,0.5455,0.551600,0.549421,0.546504,0.544978,0.534699,0.533749,0.541870,entertainment,0.176518,social,0.170545
3,https://venturebeat.com/ai/generative-ai-may-b...,generative going completely alter marketing la...,0.7521,0.7365,0.7272,0.7240,0.7216,0.732280,0.519292,0.496403,0.488997,0.482039,0.477190,0.486157,innovation,0.602014,business innovation,0.258873
4,https://www.businessinsider.com/moderna-ceo-in...,personalized cancer vaccine could provide new ...,0.5688,0.5227,0.4532,0.4527,0.4452,0.488520,0.428122,0.427320,0.422924,0.419798,0.395541,0.424541,innovation,0.430343,technology,0.140970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,https://www.cbsnews.com/news/spacex-launch-tod...,spacex falcon rocket carrying gps navigation s...,0.8183,0.4993,0.4775,0.4735,0.4719,0.480550,0.531412,0.511708,0.454913,0.436846,0.434590,0.473894,technology,0.496222,innovation,0.111382
121,https://www.cbsnews.com/video/chatgpt-jobs-art...,rise could chatgpt take job chatbots artificia...,0.7697,0.7255,0.7166,0.7103,0.6910,0.710850,0.637201,0.625964,0.625526,0.619846,0.619757,0.622773,technology,0.675250,innovation,0.216986
122,https://www.cbsnews.com/news/ukraine-military-...,jonas ohman founder ceo blue yellow lithuania ...,0.4923,0.4892,0.4892,0.4847,0.4596,0.488850,0.457436,0.457436,0.430608,0.427529,0.421351,0.438872,technology,0.349854,social,0.111065
123,https://www.cbsnews.com/news/metaverse-making-...,metaverse asset digital art video game charact...,0.6600,0.6304,0.6264,0.6229,0.6044,0.626567,0.614686,0.609527,0.597931,0.587996,0.571264,0.596281,games,0.463839,entertainment,0.240798


In [20]:
def normalize(dataset):
    dataNorm=((dataset-dataset.min())/(dataset.max()-dataset.min())) # *10
#     print (f"dataset-dataset.min() : {dataset-dataset.min()}")
#     print (f"(dataset.max()-dataset.min()): {(dataset.max()-dataset.min())}")
    print (f"dataset{dataset}", end="\n\n")
    print (f"dataNorm: {dataNorm}", end="\n\n")
#     dataNorm["article"]=dataset["article"]
#     dataNorm["summary"]=dataset["summary"]
    return dataNorm
def computeMean (df):
    AXIS = 1
    df['hfAvg'] = df[['hf1', 'hf2']].mean(axis=AXIS)
    df['kbAvg'] = df[['kb1', 'kb2']].mean(axis=AXIS)
    df['topicAvg'] = df[['score1', 'score2']].mean(axis=AXIS)
# S    print (f"df : {df}")
    return df
def displaySelectedCols(df, cols):
    return df[ cols ].copy()

def normalizeScaled(df_min_max_scaled, column): # Note must do per column!
    df_min_max_scaled[column] = (df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() - df_min_max_scaled[column].min())    
    return df_min_max_scaled

In [21]:
df2 = df[
    [
         'hf1', 'hf2', 'hf3', 'hf4', 'hf5', 'hfAvg'
        ,'kb1', 'kb2', 'kb3', 'kb4', 'kb5', 'kbAvg'
        , 'score1','score2']
        ].copy()
df2['topicAvg'] = None
# print (f"df 2 : {df2.head(20)}")
# compute mean 
df2 = computeMean(df2)
# print (f"df 3 post mean: {df3.head(20)}")
df2=normalize(df2)
df2 = df2.copy()

df2['article'] = df['article']
df2['summary'] = df['summary']
df2['topic1'] = df['topic1']
df2['topic2'] = df['topic2']

cols =  [   'article', 'summary'
       ,'hf1', 'hf2', 'hfAvg'
       ,'kb1', 'kb2', 'kbAvg'
       ,'topic1','score1','topic2','score2', 'topicAvg'
    ]
df2= displaySelectedCols(df2, cols)

pd.set_option('display.float_format','{:.2f}'.format)
df2.head(20)
display (df2)


dataset        hf1     hf2     hf3     hf4     hf5    hfAvg       kb1       kb2  \
0    0.5627  0.5556  0.5153  0.5132  0.4939  0.55915  0.408926  0.408615   
1    0.9986  0.6797  0.6685  0.6572  0.6319  0.83915  0.515490  0.456996   
2    0.5767  0.5576  0.5529  0.5504  0.5455  0.56715  0.549421  0.546504   
3    0.7521  0.7365  0.7272  0.7240  0.7216  0.74430  0.519292  0.496403   
4    0.5688  0.5227  0.4532  0.4527  0.4452  0.54575  0.428122  0.427320   
..      ...     ...     ...     ...     ...      ...       ...       ...   
120  0.8183  0.4993  0.4775  0.4735  0.4719  0.65880  0.531412  0.511708   
121  0.7697  0.7255  0.7166  0.7103  0.6910  0.74760  0.637201  0.625964   
122  0.4923  0.4892  0.4892  0.4847  0.4596  0.49075  0.457436  0.457436   
123  0.6600  0.6304  0.6264  0.6229  0.6044  0.64520  0.614686  0.609527   
124  0.5075  0.4898  0.4689  0.4566  0.4181  0.49865  0.387538  0.368311   

          kb3       kb4       kb5     kbAvg    score1    score2  topicAvg  
0   

,article,summary,hf1,hf2,hfAvg,kb1,kb2,kbAvg,topic1,score1,topic2,score2,topicAvg
0,https://datainnovation.org/2022/11/visualizing...,economist created visualization showing politi...,0.43,0.56,0.49,0.24,0.26,0.25,economics,0.64,politics,0.38,0.75
1,https://venturebeat.com/ai/stability-ai-looks-...,among organization building generative technol...,1.00,0.77,0.90,0.42,0.34,0.38,technology,0.58,innovation,0.42,0.71
2,https://venturebeat.com/ai/openai-ceo-admits-c...,watch demand session today watch herealtman la...,0.45,0.56,0.50,0.48,0.50,0.49,entertainment,0.01,social,0.39,0.03
3,https://venturebeat.com/ai/generative-ai-may-b...,generative going completely alter marketing la...,0.68,0.86,0.76,0.43,0.41,0.42,innovation,0.60,business innovation,0.63,0.86
4,https://www.businessinsider.com/moderna-ceo-in...,personalized cancer vaccine could provide new ...,0.44,0.50,0.47,0.27,0.29,0.28,innovation,0.36,technology,0.31,0.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,https://www.cbsnews.com/news/spacex-launch-tod...,spacex falcon rocket carrying gps navigation s...,0.76,0.46,0.63,0.45,0.44,0.44,technology,0.46,innovation,0.23,0.45
121,https://www.cbsnews.com/video/chatgpt-jobs-art...,rise could chatgpt take job chatbots artificia...,0.70,0.85,0.76,0.62,0.64,0.63,technology,0.70,innovation,0.51,0.91
122,https://www.cbsnews.com/news/ukraine-military-...,jonas ohman founder ceo blue yellow lithuania ...,0.34,0.44,0.39,0.32,0.35,0.33,technology,0.25,social,0.23,0.21
123,https://www.cbsnews.com/news/metaverse-making-...,metaverse asset digital art video game charact...,0.56,0.68,0.61,0.58,0.61,0.60,games,0.41,entertainment,0.58,0.61


In [22]:
# Now take a look at using min-max feature scaling
#The min-max approach (often called normalization) rescales the feature 
# to a hard and fast range of [0,1] by subtracting the minimum value of the feature 
# then dividing by the range. We can apply the min-max scaling in Pandas using the .min() and .max() methods.
columns = [
         'hf1', 'hf2', 'hf3', 'hf4', 'hf5', 'hfAvg'
        ,'kb1', 'kb2', 'kb3', 'kb4', 'kb5', 'kbAvg'
        , 'score1','score2']
df_min_max_scaled = df[ columns ].copy()
df_min_max_scaled['topicAvg'] = None
# display(df)
# apply normalization techniques by Column 1
column = 'hf1'
df_min_max_scaled= normalizeScaled(df_min_max_scaled,column)
column = 'hf2' 
df_min_max_scaled= normalizeScaled(df_min_max_scaled,column)
column = 'kb1' 
df_min_max_scaled= normalizeScaled(df_min_max_scaled,column)
column = 'kb2' 
df_min_max_scaled= normalizeScaled(df_min_max_scaled,column)

df_min_max_scaled = computeMean(df_min_max_scaled)

# view normalized data
# display(df_min_max_scaled)
# ###
df_min_max_scaled['article'] = df['article']
df_min_max_scaled['summary'] = df['summary']
df_min_max_scaled['topic1'] = df['topic1']
df_min_max_scaled['topic2'] = df['topic2']

cols =  [   'article', 'summary'
       ,'hf1', 'hf2', 'hfAvg'
       ,'kb1', 'kb2', 'kbAvg'
       ,'topic1','score1','topic2','score2', 'topicAvg'
    ]
df_min_max_scaled= displaySelectedCols(df_min_max_scaled, cols)

pd.set_option('display.float_format','{:.2f}'.format)
df_min_max_scaled.head(20)
# display (df2)


,article,summary,hf1,hf2,hfAvg,kb1,kb2,kbAvg,topic1,score1,topic2,score2,topicAvg
0,https://datainnovation.org/2022/11/visualizing...,economist created visualization showing politi...,0.43,0.56,0.49,0.24,0.26,0.25,economics,0.63,politics,0.17,0.40
1,https://venturebeat.com/ai/stability-ai-looks-...,among organization building generative technol...,1.00,0.77,0.88,0.42,0.34,0.38,technology,0.59,innovation,0.18,0.38
2,https://venturebeat.com/ai/openai-ceo-admits-c...,watch demand session today watch herealtman la...,0.45,0.56,0.51,0.48,0.50,0.49,entertainment,0.18,social,0.17,0.17
3,https://venturebeat.com/ai/generative-ai-may-b...,generative going completely alter marketing la...,0.68,0.86,0.77,0.43,0.41,0.42,innovation,0.60,business innovation,0.26,0.43
4,https://www.businessinsider.com/moderna-ceo-in...,personalized cancer vaccine could provide new ...,0.44,0.50,0.47,0.27,0.29,0.28,innovation,0.43,technology,0.14,0.29
5,https://www.digitaltrends.com/space/astronaut-...,astronaut gazing international space station m...,0.40,0.37,0.39,0.28,0.26,0.27,social,0.64,technology,0.19,0.41
6,https://executivegov.com/2022/12/dod-announces...,office undersecretary acquisition sustainment ...,0.37,0.46,0.42,0.52,0.52,0.52,innovation,0.48,business innovation,0.21,0.34
7,https://gcn.com/cloud-infrastructure/2022/12/c...,cooky used way constitutes sale data ccpa cook...,0.59,0.60,0.60,0.00,0.00,0.00,technology,0.31,social,0.19,0.25
8,https://gcn.com/cloud-infrastructure/2022/12/f...,suggestion fcc could adopt include idea like g...,0.40,0.51,0.45,0.32,0.32,0.32,social,0.40,technology,0.16,0.28
9,https://gcn.com/cloud-infrastructure/2022/12/h...,begin paying million technical debt colorado s...,0.31,0.41,0.36,0.31,0.27,0.29,technology,0.24,social,0.20,0.22


In [ ]:
#Candidates in common:: Which techniques commonly select the same articles?
TECHLABEL='technology'
INNOLABEL='innovation'
BUSILABEL='business innovation'
SCIXLABEL= 'science'
dfClassifier = df_min_max_scaled.copy()
newdf = dfClassifier.loc[
    (
        (
        ((dfClassifier['topic1']==TECHLABEL) & (dfClassifier['topic2']==INNOLABEL))  |
        ((dfClassifier['topic1']==TECHLABEL) & (dfClassifier['topic2']==BUSILABEL))  |
        ((dfClassifier['topic1']==TECHLABEL) & (dfClassifier['topic2']==SCIXLABEL))  |
        ((dfClassifier['topic1']==BUSILABEL) & (dfClassifier['topic2']==TECHLABEL))  |
        ((dfClassifier['topic1']==SCIXLABEL) & (dfClassifier['topic2']==TECHLABEL))  |        
        ((dfClassifier['topic1']==INNOLABEL) & (dfClassifier['topic2']==TECHLABEL)) 
            &
        (dfClassifier['topicAvg'] >= 0.3) 
            &
        (dfClassifier['hfAvg'] >= 0.5)
            & 
        (dfClassifier['kbAvg'] >= 0.4)
        
        ) 
    )
        , ['article','hfAvg','kbAvg','topic1','topic2', 'topicAvg']
    ]

print (f"Common Selections : {len(newdf)}")
newdf


### Test Study - 2/3/23
Wanted to compare the number of common selctions of candidate articles to the number of common selections when using the mixed-summaries repository 

In [ ]:
#Candidates in common:: Which techniques commonly select the same articles?

dfClassifier2 = df_min_max_scaled.copy()
newdf2 = dfClassifier2.loc[
    (
        (dfClassifier2['hfAvg'] >= 0.5)
            & 
        (dfClassifier2['kbAvg'] >= 0.4)
        
        ) , ['article', 'summary', 'hfAvg', 'kbAvg']
    ]

print (f"Common Selections : {len(newdf2)}")
newdf2.head(10)

In [ ]:
# Calculate percentage of candidates that were included
percent_select = (len(newdf2)/len(df))*100

print (f"Percent Selected : {percent_select}%")

### Can we build someting to test an arbitrary site to see how it scores

In [ ]:
### Test an arbitrary site to see how it scores
tsite = "https://www.cnn.com/2023/02/07/us/memphis-city-council-tyre-nichols"
elements =  extractArticle(site_url=tsite)


In [ ]:
elements['summary']